<a href="https://colab.research.google.com/github/Carol-0710/home/blob/main/EX05_04_Reply_Template_Messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q pyngrok flask_ngrok
!pip install -q flask pyngrok flask_ngrok line-bot-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.1/776.1 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 12.9 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
def ngrok_start():
    ngrok.set_auth_token(userdata.get('NGROK_AUTHTOKEN'))
    ngrok.connect(5000)
    run_with_ngrok(app)


from flask import Flask, request, abort

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import (
    Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest,
    TextMessage
)

app = Flask(__name__)

configuration = Configuration(access_token=userdata.get('LINE_CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(userdata.get('LINE_CHANNEL_SECRET'))

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

import google.generativeai as genai

genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))
model = genai.GenerativeModel("gemini-2.0-flash")
def ask_gemini(question):
    response = model.generate_content(question)
    return response.text

@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        action = event.message. text
        if action == 'confirm':
          reply = TemplateMessage(
            alt_text="早安你好呀",
            template=ConfirmTemplate(
              text="你喜歡神奇寶貝裡的水躍魚嗎?",
              actions=[
                MessageAction(label="是", text="我超喜歡!"),
                MessageAction(label="否", text="我不太熟悉。")
              ]
            )
          )

        elif action == 'carousel':
          carousel_template = CarouselTemplate(
            columns=[
              CarouselColumn(
                thumbnail_image_url='https://upload.wikimedia.org/wikipedia/zh/5/50/Pokemon_the_first_movie.jpg?20081213142208',
                title='神奇寶貝劇場版：超夢的逆襲',
                text='是神奇寶貝動畫的第一部、同時也是關都聯盟唯一的電影版',
                actions=[
                    URIAction(label='維基百科',uri='https://zh.wikipedia.org/zh-tw/%E7%A5%9E%E5%A5%87%E5%AF%B6%E8%B2%9D%E5%8A%87%E5%A0%B4%E7%89%88%EF%BC%9A%E8%B6%85%E5%A4%A2%E7%9A%84%E9%80%86%E8%A5%B2'),
                    MessageAction(label="投票",text="我投超夢的逆襲一票!")
                ]
              ),
              CarouselColumn(
                thumbnail_image_url='https://upload.wikimedia.org/wikipedia/zh/e/e8/Pokemon_the_Movie_20_poster.jpg',
                title='就決定是你了！',
                text='紀念精靈寶可夢系列20週年的作品',
                actions=[
                    URIAction(label='維基百科',uri='https://zh.wikipedia.org/zh-tw/%E5%8A%87%E5%A0%B4%E7%89%88_%E5%AF%B6%E5%8F%AF%E5%A4%A2_%E5%B0%B1%E6%B1%BA%E5%AE%9A%E6%98%AF%E4%BD%A0%E4%BA%86%EF%BC%81'),
                    MessageAction(label="投票",text="我投就決定是你了!一票!")
                ]
              )
            ]
          )


          reply = TemplateMessage(
            alt_text="這是輪播視窗",
            template=carousel_template
          )

        else:
          reply = TextMessage(text='Thanks!')

        line_bot_api.reply_message(
          ReplyMessageRequest(
            reply_token=event.reply_token,
            messages=[
              reply
            ]
          )
        )

ngrok_start() # for colab
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://f02a-34-169-182-32.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040
